# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [36]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [37]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [38]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [39]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [40]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [41]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [42]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
# YOUR CODE HERE!

k = range (1,11)

In [43]:
# Create an empy list to store the inertia values
# YOUR CODE HERE!
inertia = []

In [44]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
# YOUR CODE HERE!
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)
    
inertia

[287.0,
 195.82021818036043,
 123.19048183836959,
 79.02243535120978,
 65.30237914162501,
 52.88851821977533,
 43.91469044757748,
 37.51703249010357,
 32.48524083218354,
 28.222899290608932]

In [45]:
# Create a dictionary with the data to plot the Elbow curve
# YOUR CODE HERE!
elbow_data = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
# YOUR CODE HERE!
df_elbow_data = pd.DataFrame(elbow_data)

In [46]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# YOUR CODE HERE!

df_elbow_data.hvplot.line(
    x="k", 
    y= "inertia", 
    title="Crypto Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** According to the elbow curve, it would appear that 4 would be the best value for k

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the original data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [47]:
# Initialize the K-Means model using the best value for k
# YOUR CODE HERE!
model_4 = KMeans(n_clusters=4,random_state=0)


In [48]:
# Fit the K-Means model using the scaled data
# YOUR CODE HERE!
model_4.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=0)

In [49]:
# Predict the clusters to group the cryptocurrencies using the scaled data
# YOUR CODE HERE!
segment_4 = model_4.predict(df_market_data_scaled)
# View the resulting array of cluster values.
# YOUR CODE HERE!
segment_4

array([0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3, 3, 0, 3, 3, 0,
       3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 0, 3, 3, 2, 3, 3, 3, 3],
      dtype=int32)

In [50]:
# Add a new column to the DataFrame with the predicted clusters
# YOUR CODE HERE!
df_market_data_scaled['segment_4'] =segment_4
# Display sample data
# YOUR CODE HERE!
df_market_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [51]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
# YOUR CODE HERE!
df_market_data_scaled.hvplot.scatter(
x="price_change_percentage_24h",
y="price_change_percentage_7d",
hover_cols = 'coin_id',
by ="segment_4",
title = "Crypto Price Change Percentage 24h"

)

:NdOverlay   [segment_4]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [52]:
# Create a PCA model instance and set `n_components=3`.
# YOUR CODE HERE!
pca = PCA (n_components=3)

In [53]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
# YOUR CODE HERE!
market_pca = pca.fit_transform(df_market_data_scaled)
# View the first five rows of the DataFrame. 
# YOUR CODE HERE!
market_pca[:5]

array([[ 2.03409288, -0.64157137, -0.35762863],
       [ 1.99450168, -0.49054937, -0.91224962],
       [-1.06854412, -0.41720452,  0.5963108 ],
       [-1.04091865, -0.45523427,  0.4678008 ],
       [ 3.30276384, -1.24245766, -0.217276  ]])

In [54]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
# YOUR CODE HERE!
pca.explained_variance_ratio_

array([0.36941241, 0.29169375, 0.22893878])

#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is 87%

In [55]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you
df_crypto_pca = pd.DataFrame(market_pca, columns=["PC1", "PC2", "PC3"])
# Creating a DataFrame with the PCA data
# YOUR CODE HERE!

# Copy the crypto names from the original data
# YOUR CODE HERE!
df_crypto_pca['coin_id'] = df_market_data_scaled.index
# Set the coinid column as index
# YOUR CODE HERE!
df_crypto_pca = df_crypto_pca.set_index("coin_id")
# Display sample data
# YOUR CODE HERE!
df_crypto_pca.head(5)

,PC1,PC2,PC3
coin_id,,,
bitcoin,2.034093,-0.641571,-0.357629
ethereum,1.994502,-0.490549,-0.912250
tether,-1.068544,-0.417205,0.596311
ripple,-1.040919,-0.455234,0.467801
bitcoin-cash,3.302764,-1.242458,-0.217276


---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [56]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
# YOUR CODE HERE!

k =range(1,11)

In [57]:
# Create an empy list to store the inertia values
# YOUR CODE HERE!
inertia = []

In [58]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
# YOUR CODE HERE!

for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_crypto_pca)
    inertia.append(model.inertia_)
    
inertia

[325.7347395785631,
 201.2876224565443,
 111.73680938677688,
 40.19728442405938,
 29.94590949973295,
 20.988916219540855,
 15.3650243568266,
 12.125400369163726,
 10.472156000037076,
 7.922170720998387]

In [59]:
# Create a dictionary with the data to plot the Elbow curve
# YOUR CODE HERE!}
elbow_data = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
# YOUR CODE HERE!
df_elbow_data = pd.DataFrame(elbow_data)

In [60]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# YOUR CODE HERE!

df_elbow_data.hvplot.line(
    x="k", 
    y= "inertia", 
    title="PCA Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The Best value for k would be 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It would appear that is does not differ for the original data.

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [61]:
# Initialize the K-Means model using the best value for k
# YOUR CODE HERE!

pca_model = KMeans(n_clusters=4)

In [62]:
# Fit the K-Means model using the PCA data
# YOUR CODE HERE!
pca_model.fit(df_crypto_pca)

KMeans(n_clusters=4)

In [63]:
# Predict the clusters to group the cryptocurrencies using the PCA data
# YOUR CODE HERE!
stock_clusters = model.predict(df_crypto_pca)
# View the resulting array of cluster values.
# YOUR CODE HERE!)
stock_clusters

array([6, 6, 0, 0, 1, 1, 8, 8, 6, 0, 4, 0, 0, 1, 0, 6, 4, 0, 8, 0, 4, 6,
       0, 0, 7, 0, 0, 4, 6, 5, 0, 0, 2, 6, 9, 7, 3, 4, 0, 0, 0],
      dtype=int32)

In [64]:
# Add a new column to the DataFrame with the predicted clusters
# YOUR CODE HERE!
df_crypto_pca["PC4"] = stock_clusters
# Display sample data
# YOUR CODE HERE!
df_crypto_pca


,PC1,PC2,PC3,PC4
coin_id,,,,
bitcoin,2.034093,-0.641571,-0.357629,6
ethereum,1.994502,-0.490549,-0.912250,6
tether,-1.068544,-0.417205,0.596311,0
ripple,-1.040919,-0.455234,0.467801,0
bitcoin-cash,3.302764,-1.242458,-0.217276,1
binancecoin,2.505692,-0.575832,-0.129389,1
chainlink,2.770042,-0.495691,-1.890371,8
cardano,2.372449,-0.389146,-0.992513,8
litecoin,1.832344,-0.678797,-0.741581,6


In [69]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
# YOUR CODE HERE!

df_crypto_pca.hvplot.scatter(
    x="PC1",
    y="PC2",
    hover_cols = "coin_id",
    by = "PC4",
    title = "Scatter Plot - PCA=4"
)

:NdOverlay   [PC4]
   :Scatter   [PC1]   (PC2,coin_id)

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [73]:
# Composite plot to contrast the Elbow curves
# YOUR CODE HERE!
crypto_elbow = df_market_data_scaled.hvplot.bar(
    title ='Market Elbow Curve',
    color="blue",
    hover_color = "yellow",
    xlabel='S',
    ylabel="Total Average Loan Amount",
    rot=90
).opts(yformatter='%.0f')

In [74]:
# Compoosite plot to contrast the clusters
# YOUR CODE HERE!
#example of the other composite plot:
"""crypto_elbow = df_market_data_scaled.hvplot.bar(
    title ='Market Elbow Curve',
    color="blue",
    hover_color = "yellow",
    xlabel='State Code',
    ylabel="Total Average Loan Amount",
    rot=90
).opts(yformatter='%.0f')"""

'crypto_elbow = df_market_data_scaled.hvplot.bar(\n    title =\'Market Elbow Curve\',\n    color="blue",\n    hover_color = "yellow",\n    xlabel=\'State Code\',\n    ylabel="Total Average Loan Amount",\n    rot=90\n).opts(yformatter=\'%.0f\')'

In [72]:
#Use the '+' to bring the two composites to create a side-by-side

#composite1 + composite2

Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** YOUR ANSWER HERE!